In [1]:
import getpass
import os

if "PPLX_API_KEY" not in os.environ:
    os.environ["PPLX_API_KEY"] = getpass.getpass("Enter your Perplexity API key: ")

from langchain_core.prompts import ChatPromptTemplate
from langchain_perplexity import ChatPerplexity

chat = ChatPerplexity(temperature=0, model="sonar")

In [3]:
from langchain_core.prompts import ChatPromptTemplate

system = "You are a clear and concise assistant who explains regulatory changes."
human = (
    "Explain the change in the regulation about {regulation_topic}.\n"
    "The regulation before the change was:\n{before_text}\n\n"
    "The regulation after the change is:\n{after_text}\n\n"
    "Please describe what changed."
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", human)
])

chain = prompt | chat

response = chain.invoke({
    "regulation_topic": "data privacy",
    "before_text": "Personal data could be stored indefinitely without user consent.",
    "after_text": "Personal data must be deleted after 2 years unless consent is renewed."
})

print(response.content)

The key change in the data privacy regulation is that **personal data can no longer be stored indefinitely without user consent**; instead, **personal data must be deleted after 2 years unless the user renews their consent**. This introduces a clear time limit on data retention and requires active consent renewal to continue storing data beyond that period.

Before the change:
- Personal data could be stored indefinitely without needing to obtain or renew user consent.

After the change:
- Personal data must be deleted after 2 years.
- Continued storage beyond 2 years requires renewed user consent.

This change reflects a shift toward stronger data protection principles such as **data minimization** and **purpose limitation**, which are common in recent privacy laws taking effect in 2025 across various U.S. states. These laws emphasize limiting data retention to what is necessary and ensuring transparency and user control over personal data[1][2][3].

In summary, the regulation now enf